# Training The Transfer Learning Models
## ResNet-50 based transfer learning technique for comparison with SimpleNet
The notebook contains code used for training a transfer learning method using ResNet-50 model. 
The model classifies preprocessed retinal OCT scans into three categories: urgent referral, routine referral
and normal. **5-fold cross validation** was used to present an accurate estimate of the efficacy of the model 
on unseen data. The notebooks were run on the kaggle platform for training purposes. **Since the 5-folds
and initial weights of the deep neural net are random, the accuracies might not match with the accuracies
given in the publication.**

### Table of Contents
* [1. Importing all modules](#1)
* [2. Loading train and test data](#2)
* [3. Generator Classes using Sequence class](#3)
* [4. Functions for building and loading the transfer learning model](#4)
* [5. Functions for training and cross-validation](#5)
* [6. Training the model](#6)
* [7. Performance metrics of ResNet-50 based model](#7)

<a id="1"></a>
### 1. Importing all modules

In [1]:
import os
import keras
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense
import pandas as pd
from keras.preprocessing import image
from keras.utils import Sequence
from cv2 import * #Import functions from OpenCV
import cv2
import glob
from skimage.transform import resize
import tensorflow as tf
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from keras.models import model_from_json
import json
from statistics import mean



Using TensorFlow backend.


<a id="2"></a>
### 2. Loading train and test data

Training data is loaded from the initial k-fold split across all the four models for accurate judgement.



In [2]:
# X and y stores the images as arrays and the their target categories respectively

X = np.genfromtxt('../input/final-paper-kfold/X.out', dtype='unicode')
y = np.genfromtxt('../input/final-paper-kfold/y.out')

#reading test images, because can't use validation generator here, as that limilts the time

t_img = []
t_y=[]
for images in glob.glob("../input/no-med-filt-224/no_median_filter_224/no_median_filter_224/test/CNV/*.jpeg"):
    n = resize(imread(images), (224, 224,3))
    t_img.append(n)
    t_y.append(2)    
    
for images in glob.glob("../input/no-med-filt-224/no_median_filter_224/no_median_filter_224/test/DME/*.jpeg"):
    n = resize(imread(images), (224, 224,3))
    t_img.append(n)
    t_y.append(2)    
    
for images in glob.glob("../input/no-med-filt-224/no_median_filter_224/no_median_filter_224/test/DRUSEN/*.jpeg"):
    n = resize(imread(images), (224, 224,3))
    t_img.append(n)
    t_y.append(1)    
    
for images in glob.glob("../input/no-med-filt-224/no_median_filter_224/no_median_filter_224/test/NORMAL/*.jpeg"):
    n = resize(imread(images), (224, 224,3))
    t_img.append(n)
    t_y.append(0)    

t_img = np.asarray(t_img)
t_y = np.asarray(t_y)


<a id="3"></a>
### 3. Generator Classes using Sequence class
Since our training dataset is large, we would use the Sequence class to help us feed images in batches for training.
Different generator classes are defined for training sets and validation sets.

In [3]:
class My_Generator(Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        self.n = len(image_filenames)

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx_min = idx*self.batch_size
        idx_max = np.amin([((idx+1)*self.batch_size),self.n])
        batch_x = self.image_filenames[idx_min:idx_max]
        batch_y = self.labels[idx_min:idx_max]
        X = np.array([
            resize(imread(file_name), (224, 224,3))
               for file_name in batch_x])
        y = np.array(batch_y)
        return X,y
    
class My_valid_Generator(Sequence):

    def __init__(self, image_filenames, batch_size):
        self.image_filenames = image_filenames
        self.batch_size = batch_size
        self.n = len(image_filenames)

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx_min = idx*self.batch_size
        idx_max = np.amin([((idx+1)*self.batch_size),self.n])
        batch_x = self.image_filenames[idx_min:idx_max]
        X = np.array([
            resize(imread(file_name), (224, 224,3))
               for file_name in batch_x])
        return X
    

<a id="4"></a>
### 4. Functions for building and loading the transfer learning model

For each cross-validation build_model() builds the initial ResNet-50 based model framework whereas load_ith_model() 
loads the ith cross-validation model from memory for testing and validation.  

In [4]:
# for the first 22 epochs
def build_model():
    #load resnet50 with weights
    base_model = ResNet50(weights='../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                          include_top = False, input_shape = (224,224,3))
    for layer in base_model.layers:
        # batch-normalization layers were trained on previous imagenet images
        # so we are training batch-normalization layers again for our specific dataset
        if layer.name[0:2]!='bn': 
            layer.trainable = False                                                                                                                                        
    num_classes = 3

    x = Flatten()(base_model.output)
    predictions = Dense(num_classes, activation = 'softmax')(x)

    #create graph of your new model
    model = Model(input = base_model.input, output = predictions)

    #compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


def load_ith_model(i):
    json_file = open('model-'+str(i)+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights('model-'+str(i)+'.h5')

    #compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [5]:
# this is for the next 28 epochs

def build_model_old():
    json_file = open('../input/model-'+str(i)+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights('../input/model-'+str(i)+'.h5')

    #compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def train_save_validate_old(i):
    batch_size = 64
    train_index = np.genfromtxt('../input/final-paper-kfold/train_index-'+str(i)+'.out').astype(int)
    test_index = np.genfromtxt('../input/final-paper-kfold/test_index-'+str(i)+'.out').astype(int)
    trainData = X[train_index]
    testData = X[test_index]
    trainLabels = y[train_index]
    testLabels = y[test_index]

    trainGenerator = My_Generator(trainData,trainLabels,batch_size)
    valGenerator = My_valid_Generator(testData,400)
    
    model = build_model_old()
    
    model.fit_generator(
            generator = trainGenerator,
            steps_per_epoch=(len(trainData)//batch_size),
            epochs=28, verbose = 2, class_weight = [0.70596402, 4.19022748, 0.74357918])
    
    model_json = model.to_json()
    with open('model-'+str(i)+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('model-'+str(i)+'.h5')
    
    t_y = np.argmax(testLabels,axis=1)    
    ans = np.argmax(model.predict_generator(valGenerator),axis =1)
    
    # performance metrics  
    return performance_metrics(t_y,ans)


<a id="5"></a>
### 5. Functions for training and cross-validation
For each cross-validation i, train_save_validate() trains the model, saves model-i and saves the validation performance
metrics on validation fold. 

In [6]:

def train_save_validate(i):
    batch_size = 64
    train_index = np.genfromtxt('../input/final-paper-kfold/train_index-'+str(i)+'.out').astype(int)
    test_index = np.genfromtxt('../input/final-paper-kfold/test_index-'+str(i)+'.out').astype(int)
    trainData = X[train_index]
    testData = X[test_index]
    trainLabels = y[train_index]
    testLabels = y[test_index]

    trainGenerator = My_Generator(trainData,trainLabels,batch_size)
    valGenerator = My_valid_Generator(testData,400)
    
    model = build_model()
    if i ==1:
        model.summary()
        
    model.fit_generator(
            generator = trainGenerator,
            steps_per_epoch=(len(trainData)//batch_size),
            epochs=22, verbose = 2, class_weight = [0.70596402, 4.19022748, 0.74357918])
    
    model_json = model.to_json()
    with open('model-'+str(i)+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('model-'+str(i)+'.h5')
    
    t_y = np.argmax(testLabels,axis=1)    
    ans = np.argmax(model.predict_generator(valGenerator),axis =1)
    
    # performance metrics  
    return performance_metrics(t_y,ans)

**perfromance_metrics()** derives all performance measures from the confusion matrix whereas **print_performance_average_each_model()** prints the validation metrics in a readable format.

In [7]:
def performance_metrics(test_y,ans):
    acc = accuracy_score(test_y,ans)
    print('Accuracy = ',acc)
    target_names = ['CLass Normal', 'CLass Early', 'Class Late']
    
    # computing all metrics via the confusion matrix
    cm = confusion_matrix(test_y, ans) 
    sens0 = cm[0,0]/(cm[0,0]+cm[0,1]+cm[0,2])
    spec0 = (cm[1,1]+cm[1,2]+cm[2,1] +cm[2,2])/((cm[1,1]+cm[1,2]+cm[2,1] +cm[2,2])+(cm[1,0]+cm[2,0]))

    sens1 = cm[1,1]/(cm[1,0]+cm[1,1]+cm[1,2])
    spec1 = (cm[0,0]+cm[2,0]+cm[0,2] +cm[2,2])/((cm[0,0]+cm[2,0]+cm[0,2] +cm[2,2])+(cm[0,1]+cm[2,1]))

    sens2 = cm[2,2]/(cm[2,0]+cm[2,1]+cm[2,2])
    spec2 = (cm[0,0]+cm[0,1]+cm[1,0] +cm[1,1])/((cm[0,0]+cm[0,1]+cm[1,0] +cm[1,1])+(cm[0,2]+cm[1,2]))
  
    rep=classification_report(test_y, ans, target_names=target_names, digits = 4, labels=range(0,3),output_dict=True)
    return [acc,sens0,spec0,sens1,spec1,sens2,spec2],cm,rep

# method for printing performance 
def print_performance_average_each_model(p,i):
    a = list(p[i][2].keys())
    b = list(p[i][2][a[0]].keys())

    s = np.zeros(7)
    pn = 0
    pe = 0
    pl =0
    fn =0
    fe = 0
    fl = 0
    rn = 0
    re = 0
    rl = 0
    for j in range(i,i+1):
        s = s + p[j][0]
        pn = pn +  p[j][2][a[0]][b[0]]
        pe = pe +  p[j][2][a[1]][b[0]]
        pl = pl +  p[j][2][a[2]][b[0]]
        fn = fn + p[j][2][a[0]][b[2]]
        fe = fe + p[j][2][a[1]][b[2]]
        fl = fl +  p[j][2][a[2]][b[2]]
        rn = rn +  p[j][2][a[0]][b[1]]
        re = re +  p[j][2][a[1]][b[1]]
        rl = rl +  p[j][2][a[2]][b[1]]
    print("\n\n###### Metrics for Fold ",i, " ######" )
    print('\nacc = ',round(s[0]*100,2))
    print("              sensitivity  specificity      precision      f1-score")
    print('\nClass Normal: ',round(s[1]*100,2), ",       ", round(s[2]*100,2), ",       ", round(pn*100,2),",       ", round(fn*100,2))
    print('\nClass Early:  ',round(s[3]*100,2), ",       ", round(s[4]*100,2), ",       ", round(pe*100,2),",       ", round(fe*100,2))
    print('\nClass Late:   ',round(s[5]*100,2), ",       ", round(s[6]*100,2), ",       ", round(pl*100,2),",       ", round(fl*100,2))

    print('\n\n##### Average Performance Metrics for Fold ', i, ' #####')
    avg_prec = round(p[j][2][a[5]][b[0]]*100,2)
    avg_sens = round(np.sum([s[1]*p[j][2][a[0]][b[3]],
                      s[3]*p[j][2][a[1]][b[3]],
                      s[5]*p[j][2][a[2]][b[3]]])*(100/p[j][2][a[5]][b[3]]),2)
    avg_spec = round(np.sum([s[2]*p[j][2][a[0]][b[3]],
                      s[4]*p[j][2][a[1]][b[3]],
                      s[6]*p[j][2][a[2]][b[3]]])*(100/p[j][2][a[5]][b[3]]),2)
    avg_f1 = round(p[j][2][a[5]][b[2]]*100,2)
    acc = round(s[0]*100,2)
    print('\nprecision: ',avg_prec,'sensitivity: ',avg_sens, ", specificity: ",avg_spec, ", f1-score: ", avg_f1)
    return [avg_prec, avg_sens, avg_spec, avg_f1, acc]


 <a id="6"></a>
### 6. Training the ResNet-50 model
This cell is not executed in this notebook as each training will yield different results. Kindly send me an email if you need the trained weights used in the publication.

Each cell executes a part of the cross-validation because each cell in kaggle had a maximum running time limit. This notebook shows the first 22 epochs of execution, the next 28 epochs were run on a separate notebook (with adequately modified functions to load and train the model) and is not shown due to redundancy. Kindly send me an email if you need the other notebook. If you have the computing power to run it all in one go, please change of epochs from 22 to 50 in the train_save_validate(i) method.


In [8]:
valid_perf = {}

# the first 22 epochs
for i in range(1,2):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    train_save_validate(i)


====== K Fold Validation step => 1/5 =======
Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
# the next 28 epochs
for i in range(1,2):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i]= train_save_validate_old(i)


====== K Fold Validation step => 1/5 =======
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/28
 - 1154s - loss: 1.4324 - acc: 0.9110
Epoch 2/28
 - 1039s - loss: 1.4316 - acc: 0.9111
Epoch 3/28
 - 1047s - loss: 1.4355 - acc: 0.9108
Epoch 4/28
 - 1052s - loss: 1.4300 - acc: 0.9112
Epoch 5/28
 - 1099s - loss: 1.4189 - acc: 0.9119
Epoch 6/28
 - 1040s - loss: 1.3955 - acc: 0.9133
Epoch 7/28
 - 1027s - loss: 1.4126 - acc: 0.9123
Epoch 8/28
 - 1032s - loss: 1.4049 - acc: 0.9128
Epoch 9/28
 - 1081s - loss: 1.4116 - acc: 0.9123
Epoch 10/28
 - 1034s - loss: 1.4106 - acc: 0.9124
Epoch 11/28
 - 1033s - loss: 1.4106 - acc: 0.9124
Epoch 12/28
 - 1036s - loss: 1.4017 - acc: 0.9130
Epoch 13/28
 - 1073s - loss: 1.4239 - acc: 0.9116
Epoch 14/28
 - 1035s - loss: 1.4017 - acc: 0.9130
Epoch 15/28
 - 1027s - loss: 1.4009 - acc: 0.9130
Epoch 16/28
 - 1027s - loss: 1.4096 - acc: 0.9125
Epoch 17/28
 - 1064s - loss: 1.4264 - acc: 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
# fold 2 28 epochs
for i in range(2,3):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i] = train_save_validate(i)


====== K Fold Validation step => 2/5 =======
Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Use tf.cast instead.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/28
 - 1219s - loss: 1.6423 - acc: 0.8931
Epoch 2/28
 - 1077s - loss: 1.5221 - acc: 0.9045
Epoch 3/28
 - 1097s - loss: 1.5034 - acc: 0.9062
Epoch 4/28
 - 1093s - loss: 1.4769 - acc: 0.9080
Epoch 5/28
 - 1130s - loss: 1.4698 - acc: 0.9085
Epoch 6/28
 - 1089s - loss: 1.4753 - acc: 0.9082
Epoch 7/28
 - 1079s - loss: 1.4786 - acc: 0.9080
Epoch 8/28
 - 1079s - loss: 1.4991 - acc: 0.9068
Epoch 9/28
 - 1130s - loss: 1.4617 - acc: 0.9091
Epoch 10/28
 - 1081s - loss: 1.4317 - acc: 0.9110
Epoch 11/28
 - 1076s - loss: 1.4419 - acc: 0.9105
Epoch 12/28
 - 1065s - loss: 1.4663 - acc: 0.9089
Epoch 13/28
 - 1116s - loss: 1.4329 - acc: 0.9110
Epoch 14/28
 - 1094s - loss: 1.4518 - acc: 0.9098
Epoch 15/28
 - 1140s - loss: 1.4599 - acc: 0.9093
Epoch 16/28
 - 1131s - loss: 1.4557 - acc: 0.9096
Epoch 17/28
 - 1155s - loss: 1.4490 - acc: 0.9099
Epoch 18/28
 - 1091s - loss: 1.4335 - acc: 0.9110
Epoch 19/28
 - 1078s - loss: 1.4329 - acc: 0.9110
Epoch 20/28
 - 1055s - loss: 1.4425 - acc: 0.9104
Epoch 21/

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
# fold 3 first 22 epochs
for i in range(3,4):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    train_save_validate(i)


====== K Fold Validation step => 3/5 =======
Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Use tf.cast instead.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/22
 - 1013s - loss: 1.6498 - acc: 0.8920
Epoch 2/22
 - 946s - loss: 1.5243 - acc: 0.9044
Epoch 3/22
 - 951s - loss: 1.5010 - acc: 0.9063
Epoch 4/22
 - 935s - loss: 1.4881 - acc: 0.9073
Epoch 5/22
 - 930s - loss: 1.4715 - acc: 0.9084
Epoch 6/22
 - 939s - loss: 1.4972 - acc: 0.9069
Epoch 7/22
 - 929s - loss: 1.4729 - acc: 0.9084
Epoch 8/22
 - 927s - loss: 1.4719 - acc: 0.9085
Epoch 9/22
 - 931s - loss: 1.4687 - acc: 0.9087
Epoch 10/22
 - 934s - loss: 1.4552 - acc: 0.9096
Epoch 11/22
 - 931s - loss: 1.4855 - acc: 0.9077
Epoch 12/22
 - 932s - loss: 1.4824 - acc: 0.9079
Epoch 13/22
 - 958s - loss: 1.4673 - acc: 0.9089
Epoch 14/22
 - 945s - loss: 1.4664 - acc: 0.9089
Epoch 15/22
 - 940s - loss: 1.4785 - acc: 0.9081
Epoch 16/22
 - 953s - loss: 1.4419 - acc: 0.9104
Epoch 17/22
 - 951s - loss: 1.4381 - acc: 0.9107
Epoch 18/22
 - 944s - loss: 1.4554 - acc: 0.9096
Epoch 19/22
 - 934s - loss: 1.4632 - acc: 0.9092
Epoch 20/22
 - 929s - loss: 1.4708 - acc: 0.9086
Epoch 21/22
 - 942s - loss: 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
# fold 3 next 28epochs
for i in range(3,4):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i] = train_save_validate_old(i)


====== K Fold Validation step => 3/5 =======
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/28
 - 1060s - loss: 1.4561 - acc: 0.9095
Epoch 2/28
 - 966s - loss: 1.4466 - acc: 0.9101
Epoch 3/28
 - 978s - loss: 1.4226 - acc: 0.9117
Epoch 4/28
 - 955s - loss: 1.4329 - acc: 0.9110
Epoch 5/28
 - 966s - loss: 1.4553 - acc: 0.9096
Epoch 6/28
 - 986s - loss: 1.4412 - acc: 0.9104
Epoch 7/28
 - 995s - loss: 1.4284 - acc: 0.9113
Epoch 8/28
 - 976s - loss: 1.4449 - acc: 0.9103
Epoch 9/28
 - 989s - loss: 1.4552 - acc: 0.9097
Epoch 10/28
 - 985s - loss: 1.4492 - acc: 0.9100
Epoch 11/28
 - 977s - loss: 1.4293 - acc: 0.9112
Epoch 12/28
 - 969s - loss: 1.4155 - acc: 0.9121
Epoch 13/28
 - 977s - loss: 1.4318 - acc: 0.9111
Epoch 14/28
 - 976s - loss: 1.4483 - acc: 0.9101
Epoch 15/28
 - 989s - loss: 1.4242 - acc: 0.9116
Epoch 16/28
 - 975s - loss: 1.4769 - acc: 0.9083
Epoch 17/28
 - 986s - loss: 1.4344 - acc: 0.9109
Epoch 18/

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
# fold 4 first 22 epochs
for i in range(4,5):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    train_save_validate(i,0)


====== K Fold Validation step => 4/5 =======
Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Use tf.cast instead.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/22
 - 1014s - loss: 1.6809 - acc: 0.8900
Epoch 2/22
 - 947s - loss: 1.5207 - acc: 0.9046
Epoch 3/22
 - 953s - loss: 1.5065 - acc: 0.9058
Epoch 4/22
 - 940s - loss: 1.4932 - acc: 0.9069
Epoch 5/22
 - 946s - loss: 1.4755 - acc: 0.9082
Epoch 6/22
 - 934s - loss: 1.4682 - acc: 0.9086
Epoch 7/22
 - 955s - loss: 1.4705 - acc: 0.9085
Epoch 8/22
 - 959s - loss: 1.4668 - acc: 0.9088
Epoch 9/22
 - 964s - loss: 1.4584 - acc: 0.9094
Epoch 10/22
 - 959s - loss: 1.4677 - acc: 0.9088
Epoch 11/22
 - 961s - loss: 1.4611 - acc: 0.9092
Epoch 12/22
 - 960s - loss: 1.4534 - acc: 0.9097
Epoch 13/22
 - 963s - loss: 1.4637 - acc: 0.9091
Epoch 14/22
 - 942s - loss: 1.4662 - acc: 0.9089
Epoch 15/22
 - 936s - loss: 1.4479 - acc: 0.9100
Epoch 16/22
 - 940s - loss: 1.4553 - acc: 0.9096
Epoch 17/22
 - 943s - loss: 1.4330 - acc: 0.9109
Epoch 18/22
 - 937s - loss: 1.4343 - acc: 0.9109
Epoch 19/22
 - 939s - loss: 1.4335 - acc: 0.9109
Epoch 20/22
 - 933s - loss: 1.4701 - acc: 0.9087
Epoch 21/22
 - 938s - loss: 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
# fold 3 next 28 epochs
for i in range(4,5):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i]= train_save_validate_old(i)


====== K Fold Validation step => 4/5 =======
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/28
 - 1023s - loss: 1.4282 - acc: 0.9113
Epoch 2/28
 - 933s - loss: 1.4263 - acc: 0.9115
Epoch 3/28
 - 927s - loss: 1.4284 - acc: 0.9113
Epoch 4/28
 - 924s - loss: 1.4306 - acc: 0.9112
Epoch 5/28
 - 960s - loss: 1.4451 - acc: 0.9103
Epoch 6/28
 - 924s - loss: 1.4462 - acc: 0.9102
Epoch 7/28
 - 922s - loss: 1.4234 - acc: 0.9117
Epoch 8/28
 - 921s - loss: 1.4262 - acc: 0.9115
Epoch 9/28
 - 958s - loss: 1.4852 - acc: 0.9078
Epoch 10/28
 - 922s - loss: 1.4587 - acc: 0.9094
Epoch 11/28
 - 923s - loss: 1.4169 - acc: 0.9121
Epoch 12/28
 - 923s - loss: 1.4345 - acc: 0.9109
Epoch 13/28
 - 957s - loss: 1.4291 - acc: 0.9113
Epoch 14/28
 - 917s - loss: 1.4391 - acc: 0.9106
Epoch 15/28
 - 916s - loss: 1.4495 - acc: 0.9100
Epoch 16/28
 - 941s - loss: 1.4166 - acc: 0.9120
Epoch 17/28
 - 996s - loss: 1.4372 - acc: 0.9108
Epoch 18/

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
#fold 5 first 22 epochs
for i in range(5,6):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i] = train_save_validate(i)


====== K Fold Validation step => 5/5 =======
Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Use tf.cast instead.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/22
 - 1370s - loss: 1.6348 - acc: 0.8934
Epoch 2/22
 - 1251s - loss: 1.5146 - acc: 0.9049
Epoch 3/22
 - 1253s - loss: 1.4831 - acc: 0.9074
Epoch 4/22
 - 1243s - loss: 1.4753 - acc: 0.9081
Epoch 5/22
 - 1300s - loss: 1.4823 - acc: 0.9077
Epoch 6/22
 - 1253s - loss: 1.4743 - acc: 0.9083
Epoch 7/22
 - 1235s - loss: 1.4903 - acc: 0.9073
Epoch 8/22
 - 1227s - loss: 1.4500 - acc: 0.9098
Epoch 9/22
 - 1292s - loss: 1.4449 - acc: 0.9103
Epoch 10/22
 - 1276s - loss: 1.4561 - acc: 0.9095
Epoch 11/22
 - 1265s - loss: 1.4367 - acc: 0.9107
Epoch 12/22
 - 1259s - loss: 1.4346 - acc: 0.9108
Epoch 13/22
 - 1303s - loss: 1.4414 - acc: 0.9104
Epoch 14/22
 - 1263s - loss: 1.4345 - acc: 0.9109
Epoch 15/22
 - 1250s - loss: 1.4333 - acc: 0.9110
Epoch 16/22
 - 1140s - loss: 1.4917 - acc: 0.9073
Epoch 17/22
 - 1139s - loss: 1.4500 - acc: 0.9099
Epoch 18/22
 - 1094s - loss: 1.4263 - acc: 0.9114
Epoch 19/22
 - 1114s - loss: 1.4596 - acc: 0.9093
Epoch 20/22
 - 1188s - loss: 1.4166 - acc: 0.9120
Epoch 21/

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


<a id="7"></a>
### 7. Performance metrics of ResNet-50 based model

In [16]:
# all performance metrics for each fold and their averages
perf = np.zeros(5)

for i in range(1,6):
    perf += print_performance_average_each_model(valid_perf,i)

perf = perf/5

print('\n#### AVERAGE PERFORMANCE OVER ALL FOLDS ####\n\n\nprecision: ',round(perf[0],2),'sensitivity: ',round(perf[1],2), ", specificity: ",round(perf[2],2), ", f1-score: ", round(perf[3],2))   
print('\n[',round(perf[0],2),', ',round(perf[1],2), ", ",round(perf[2],2), ", ", round(perf[3],2),"]")   
print('\n overall accuracy of model = ', round(perf[4],2))



###### Metrics for Fold  1  ######

acc =  90.82
              sensitivity  specificity      precision      f1-score

Class Normal:  99.74 ,        91.68 ,        91.35 ,        95.36

Class Early:   0.0 ,        100.0 ,        0.0 ,        0.0

Class Late:    97.44 ,        91.31 ,        90.27 ,        93.72


##### Average Performance Metrics for Fold  1  #####

precision:  83.65 sensitivity:  90.82 , specificity:  92.17 , f1-score:  87.09


###### Metrics for Fold  3  ######

acc =  90.56
              sensitivity  specificity      precision      f1-score

Class Normal:  99.12 ,        92.97 ,        92.7 ,        95.81

Class Early:   0.0 ,        100.0 ,        0.0 ,        0.0

Class Late:    97.62 ,        89.63 ,        88.37 ,        92.76


##### Average Performance Metrics for Fold  3  #####

precision:  83.39 sensitivity:  90.56 , specificity:  92.04 , f1-score:  86.82


###### Metrics for Fold  4  ######

acc =  91.32
              sensitivity  specificity      precisio